- ./bin/sample_service.py # run service
- locust -f benchmark/locustfile.py --host=http://127.0.0.1:8890  # run locust
- open web ui http://127.0.0.1:8089/

docs at https://docs.locust.io/en/stable/quickstart.html

- `locust -f benchmark/sleep50_locust.py --master --host=http://127.0.0.1:8890`
- `locust -f benchmark/sleep50_locust.py --slave --host=http://127.0.0.1:8890`


go


```
locust -f benchmark/dummy.py --master --master-bind-host=127.0.0.1 --master-bind-port=5557

cd benchmark/boomer
go build -o a.out http_benchmark.go

./a.out --url=http://127.0.0.1:8890/sleep50 --master-port=5557 --rpc=zeromq
```

In [1]:
run_locust_master = 'locust -f dummy.py --master --master-bind-host=127.0.0.1 --master-bind-port=5557 --csv=foobar --no-web -c 100 -r 100 -n 1000'
run_locust_slave = './a.out --url=http://127.0.0.1:8890/sleep50 --master-port=5557 --rpc=zeromq'

In [2]:
import os

In [3]:
os.system('./run_locust.sh 10')

0

In [4]:
import pandas as pd

In [5]:
pd.read_csv('foobar_distribution.csv')

,Name,# requests,50%,66%,75%,80%,90%,95%,98%,99%,100%
0,http 200,3280,53,54,54,54,55,56,56,57,59
1,None Total,3280,53,54,54,54,55,56,56,57,59


In [6]:
pd.read_csv('foobar_requests.csv')

,Method,Name,# requests,# failures,Median response time,Average response time,Min response time,Max response time,Average Content Size,Requests/s
0,http,200,3280,0,53,52,50,59,2,207.57
1,None,Total,3280,0,53,52,50,59,2,207.57


In [62]:
import tqdm
import time

In [74]:
def run_experiment(rps: list, users: list, duration_sec=7):
    all_distributions = []
    all_requests = []
    
    for rp, users in tqdm.tqdm(zip(rps, users)):        
        #users = int(rps / 10 + 2)
        #users = 5
        print(rp, users)
        os.system(f'./run_locust.sh {users} {duration_sec} {rp}')
        time.sleep(1)
        distribution = pd.read_csv('foobar_distribution.csv')
        distribution['rps'] = rp
        all_distributions.append(distribution)
        requests = pd.read_csv('foobar_requests.csv')
        requests['rps'] = rp
        all_requests.append(requests)
        print(requests)
        
    distributions = pd.concat(all_distributions)
    distributions = distributions[distributions.Name != 'None Total']
    
    requests = pd.concat(all_requests)
    requests = requests[requests.Name != 'Total']
    return distributions, requests

In [75]:
def report(requests, distributions, experiment):
    return pd.DataFrame({
        'experiment': experiment,
        #'me': requests['Name'],
        'rps': requests['Requests/s'],
        'load': requests['rps'],
        '50': distributions['50%'],
        '75': distributions['75%'],
        '95': distributions['95%'],
        '99': distributions['99%'],
        '100': distributions['100%'],
    })

In [76]:
distributions, requests = run_experiment(users=[10, 20, 30], rps=[100, 200, 300], duration_sec=10)
r = report(requests, distributions, 'aiohttp uvloop')
r


0it [00:00, ?it/s]

100 10



1it [00:11, 11.55s/it]

  Method   Name  # requests  # failures  Median response time  \
0   http    200         809           0                    54   
1   None  Total         809           0                    54   

   Average response time  Min response time  Max response time  \
0                     53                 50                 60   
1                     53                 50                 60   

   Average Content Size  Requests/s  rps  
0                     2      134.46  100  
1                     2      134.46  100  
200 20



2it [00:23, 11.54s/it]

  Method   Name  # requests  # failures  Median response time  \
0   http    200         809           0                    54   
1   None  Total         809           0                    54   

   Average response time  Min response time  Max response time  \
0                     53                 50                 60   
1                     53                 50                 60   

   Average Content Size  Requests/s  rps  
0                     2      134.46  200  
1                     2      134.46  200  
300 30



3it [00:34, 11.54s/it]


  Method   Name  # requests  # failures  Median response time  \
0   http    200         809           0                    54   
1   None  Total         809           0                    54   

   Average response time  Min response time  Max response time  \
0                     53                 50                 60   
1                     53                 50                 60   

   Average Content Size  Requests/s  rps  
0                     2      134.46  300  
1                     2      134.46  300  


,experiment,rps,load,50,75,95,99,100
0,aiohttp uvloop,134.46,100,54,55,57,58,60
0,aiohttp uvloop,134.46,200,54,55,57,58,60
0,aiohttp uvloop,134.46,300,54,55,57,58,60


In [77]:
distributions, requests = run_experiment(users=[50], rps=[1000], duration_sec=10)
r2 = report(requests, distributions, 'aiohttp uvloop')
r2


0it [00:00, ?it/s]

1000 50



1it [00:11, 11.53s/it]


  Method   Name  # requests  # failures  Median response time  \
0   http    200         809           0                    54   
1   None  Total         809           0                    54   

   Average response time  Min response time  Max response time  \
0                     53                 50                 60   
1                     53                 50                 60   

   Average Content Size  Requests/s   rps  
0                     2      134.46  1000  
1                     2      134.46  1000  


,experiment,rps,load,50,75,95,99,100
0,aiohttp uvloop,134.46,1000,54,55,57,58,60


In [42]:
requests

,Method,Name,# requests,# failures,Median response time,Average response time,Min response time,Max response time,Average Content Size,Requests/s,rps
0,http,200 http,25,0,54,54,51,57,2,25.0,5
0,http,200 http,25,0,54,54,51,57,2,25.0,10


In [34]:
distributions

,Name,# requests,50%,66%,75%,80%,90%,95%,98%,99%,100%,rps
0,http 200,25,54,55,56,56,56,57,57,57,57,5
0,http 200,25,54,55,56,56,56,57,57,57,57,10
